# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

## Подготовка данных

In [1]:
import pandas as pd
from sklearn.utils import shuffle
from sklearn.metrics import f1_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('/datasets/Churn.csv')

In [3]:
print(df.info)
df = df.dropna()

<bound method DataFrame.info of       RowNumber  CustomerId    Surname  CreditScore Geography  Gender  Age  \
0             1    15634602   Hargrave          619    France  Female   42   
1             2    15647311       Hill          608     Spain  Female   41   
2             3    15619304       Onio          502    France  Female   42   
3             4    15701354       Boni          699    France  Female   39   
4             5    15737888   Mitchell          850     Spain  Female   43   
...         ...         ...        ...          ...       ...     ...  ...   
9995       9996    15606229   Obijiaku          771    France    Male   39   
9996       9997    15569892  Johnstone          516    France    Male   35   
9997       9998    15584532        Liu          709    France  Female   36   
9998       9999    15682355  Sabbatini          772   Germany    Male   42   
9999      10000    15628319     Walker          792    France  Female   28   

      Tenure    Balance  NumOfP

Забыл закомментить, я это сделал чтобы в кодах не вылазело ошибок, потому что во время написания одного из циклов питон выдавал ошибку, связанную с этим

In [4]:
print(df.head())
print(df['Geography'].unique())

   RowNumber  CustomerId   Surname  CreditScore Geography  Gender  Age  \
0          1    15634602  Hargrave          619    France  Female   42   
1          2    15647311      Hill          608     Spain  Female   41   
2          3    15619304      Onio          502    France  Female   42   
3          4    15701354      Boni          699    France  Female   39   
4          5    15737888  Mitchell          850     Spain  Female   43   

   Tenure    Balance  NumOfProducts  HasCrCard  IsActiveMember  \
0     2.0       0.00              1          1               1   
1     1.0   83807.86              1          0               1   
2     8.0  159660.80              3          1               0   
3     1.0       0.00              2          0               0   
4     2.0  125510.82              1          1               1   

   EstimatedSalary  Exited  
0        101348.88       1  
1        112542.58       0  
2        113931.57       1  
3         93826.63       0  
4         790

In [5]:
df = pd.get_dummies(data=df, columns=['Gender', 'Geography'], drop_first=True)

Есть очень много столбцов, которые никак не важны для обучения модели, поэтому попробуем их удалить, ибо никакой значимой для нас информации они не несут. Также есть некоторые столбцы, которые необходимо перевоплотить в другой формат, чтобы их можно было использовать в обучении, например столбец страны стоит превратить в 0, 1, 2, а гендер стоит превратить в 0 и 1 соответственно

In [6]:
df = df.drop(['RowNumber', 'CustomerId', 'Surname'], axis  = 1)

In [7]:
df.describe()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Gender_Male,Geography_Germany,Geography_Spain
count,9091.000000,9091.000000,9091.000000,9091.000000,9091.000000,9091.000000,9091.000000,9091.000000,9091.000000,9091.000000,9091.000000,9091.000000
mean,650.736553,38.949181,4.997690,76522.740015,1.530195,0.704983,0.515565,100181.214924,0.203938,0.547135,0.252227,0.247278
std,96.410471,10.555581,2.894723,62329.528576,0.581003,0.456076,0.499785,57624.755647,0.402946,0.497801,0.434315,0.431453
min,350.000000,18.000000,0.000000,0.000000,1.000000,0.000000,0.000000,11.580000,0.000000,0.000000,0.000000,0.000000
25%,584.000000,32.000000,2.000000,0.000000,1.000000,0.000000,0.000000,51227.745000,0.000000,0.000000,0.000000,0.000000
50%,652.000000,37.000000,5.000000,97318.250000,1.000000,1.000000,1.000000,100240.200000,0.000000,1.000000,0.000000,0.000000
75%,717.000000,44.000000,7.000000,127561.890000,2.000000,1.000000,1.000000,149567.210000,0.000000,1.000000,1.000000,0.000000
max,850.000000,92.000000,10.000000,250898.090000,4.000000,1.000000,1.000000,199992.480000,1.000000,1.000000,1.000000,1.000000


По результатам предварительного осмотра данных не было замечено каких то явных отклонений, единственное что - в нашей таргет переменной всего 20% единиц, что указывает на дисбаланс классов. Удалено было всего 10% значений, что не является сильно значимым в данном случае

## Исследование задачи

Для начала построю модель без учет аклассов и посмотрим метрику для лучшей из моделей

In [8]:
train, data1 = train_test_split(df, test_size=0.4, random_state=12345)
target_train = train['Exited']
features_train = train.drop(['Exited'] , axis=1)
target = data1['Exited']
features = data1.drop(['Exited'] , axis=1)
features_test, features_valid, target_test, target_valid = train_test_split(features, target, test_size=0.5, random_state=12345)

Перебил на 3 выборки, 0,6 0,2 и 0,2.

In [9]:
best_model = None
best_f1 = 0.01
best_depth = 0
for est in range(1, 50, 2):
    for depth in range(1, 12):
        model = RandomForestClassifier(n_estimators = est, max_depth=depth, random_state=12345)
        model.fit(features_train, target_train)
        result = model.score(features_valid, target_valid)
        predicted_valid = model.predict(features_valid)
        result_f1 = f1_score(target_valid, predicted_valid)
        if result_f1 > best_f1:
            best_model = model
            best_f1 = result_f1
            best_depth = depth
            best_est = est
print(best_f1, best_depth, best_est, best_model)

0.572463768115942 10 49 RandomForestClassifier(max_depth=10, n_estimators=49, random_state=12345)


Получилась достатончо низкая Ф1 мера, всего 0,57, конечно для проверки недостает всего 0,01, но так или иначе если модель будет каждый раз выбирать 0 точность будет выше, поэтому порпобуем улучшить точноость модели поменяв абаланс калссов

## Борьба с дисбалансом

Для начала попробуем просто добавить параметр class_weight для нашей модели, опять перебрав все модели и выбрав лучшую

In [10]:
for est in range(1, 100, 2):
    for depth in range(1, 15):
        model = RandomForestClassifier(n_estimators = est, max_depth=depth, random_state=12345, class_weight = 'balanced')
        model.fit(features_train, target_train)
        result = model.score(features_valid, target_valid)
        predicted_valid = model.predict(features_valid)
        result_f1 = f1_score(target_valid, predicted_valid)
        if result_f1 > best_f1:
            best_model = model
            best_f1 = result_f1
            best_depth = depth
            best_est = est
print(best_f1, best_depth, best_est, best_model)

0.6255924170616114 12 87 RandomForestClassifier(class_weight='balanced', max_depth=12, n_estimators=87,
                       random_state=12345)


Отлично, мера повысилась аж почти на 0,04 и уже можно точность получилась выше минимально допустимой,  но попробуем еще, применив другие методы борьбы с неравенством классов, например метод апсемплинга

In [11]:
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled

features_upsampled, target_upsampled = upsample(features_train, target_train, 4)

best_f1 = 0.01
for est in range(1, 32, 2):
    for depth in range(1, 13):
            for criteria in ['gini', 'entropy']:
                model = RandomForestClassifier(n_estimators = est, max_depth=depth, random_state=12345, criterion = criteria)
                model.fit(features_upsampled, target_upsampled)
                predicted_valid = model.predict(features_valid)
                result_f1 = f1_score(target_valid, predicted_valid)
                if result_f1 > best_f1:
                    best_model = model
                    best_f1 = result_f1
print(best_f1, best_model)

0.6033810143042913 RandomForestClassifier(criterion='entropy', max_depth=11, n_estimators=23,
                       random_state=12345)


0,60 без апсемплинга была выше, но оно вполне ожидаемо, зато так будет гораздо лучше определать единицы

In [12]:
target_upsampled.describe()

count    8832.000000
mean        0.509964
std         0.499929
min         0.000000
25%         0.000000
50%         1.000000
75%         1.000000
max         1.000000
Name: Exited, dtype: float64

## Тестирование модели

In [13]:
predicted_test = best_model.predict(features_test)
result_f1 = f1_score(target_test, predicted_test)

Cделал просто апсемплинг, так как мне очень нравится этот способ борьбы с неравенством классов

Результат получился немного ниже чем был на последней моделе но он все равно допустимый. Скорее всего это проихошло потому что в прошлый раз модель научилась очень хорошо определять нули и очень плохо единицы, сейчас же она научилась хорошо определять нули и также хорошо определять единицы, так или иначе рузультат оставлет желать лучшего.

Теперь посмотрим метрику AUC-ROC

In [15]:
probabilities_valid = best_model.predict_proba(features_test)
probabilities_one_valid = probabilities_valid[:, 1]

auc_roc = roc_auc_score(target_test, probabilities_one_valid)
print(auc_roc)

0.8588651411762758


Качество данной метрики уже очень даже хорошее, выше случайно модели аж на 0,35, что является вполне достойным значением

В заключение стоит сказать, что в результате наша модель научилась достаточно хорошо определять нули и единицы в равной степени, метрика Ф1 получилась равной 0,60, что в целом допустимо. Также имеется очень высокий результат касательно AUC-ROC метрики, целых 0,85. Также в течение построения моделей были применяны разные методы борьбы с неравенством классов, один из них - это дополнительный параметр в моделе, другой - апсемплинг